In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2024/Conferences.csv
/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv
/kaggle/input/march-machine-learning-mania-2024/Cities.csv
/

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

2024-04-02 03:20:32.499778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 03:20:32.499935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 03:20:32.667118: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Teams and Seasons

In [3]:
teams_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeams.csv")
teams_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WTeams.csv")

seasons_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv")
seasons_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WSeasons.csv")

### Tournament Seeds and Results

In [4]:
tourney_seeds_2024 = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv")

tournament_seeds_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeeds.csv")
tournament_seeds_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySeeds.csv")

season_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonCompactResults.csv")
season_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv")

tournament_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv")
tournament_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyCompactResults.csv")

### Detailed Game Results

In [5]:
detailed_season_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv")
detailed_season_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonDetailedResults.csv")
detailed_tournament_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyDetailedResults.csv")
detailed_tournament_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv")

### Geography

In [6]:
game_cities = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/Cities.csv")
game_cities_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv")
game_cities_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv")

### Public Rankings

In [7]:
mens_ranking = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MMasseyOrdinals_thruSeason2024_day128.csv")

### Supplemental Data

In [8]:
conference = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/Conferences.csv")
conference_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeamConferences.csv")
conference_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WTeamConferences.csv")
conference_tournaments_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MConferenceTourneyGames.csv")

team_names_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeamSpellings.csv",encoding='cp1252')
team_names_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WTeamSpellings.csv",encoding='cp1252')
team_coaches_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeamCoaches.csv")

tournament_seed_round_slots_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv")

tournament_slots_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySlots.csv")
tournament_slots_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv")

secondary_tournament_teams_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv")
secondary_tournaments_results_men= pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyCompactResults.csv",encoding='cp1252')

sample_submission = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv")


### Feature Selection

In [9]:
combined_season_results = pd.concat([season_results_men, season_results_women])

# Calculate win rates and score margins for women and men
combined_season_results['win'] = np.ones(len(combined_season_results))
win_rates = combined_season_results.groupby('WTeamID').size() / (combined_season_results.groupby('WTeamID').size() + combined_season_results.groupby('LTeamID').size())
win_rates = win_rates.reset_index(name='WinRate')

combined_season_results['ScoreMargin'] = combined_season_results['WScore'] - combined_season_results['LScore']
avg_score_margin= combined_season_results.groupby('WTeamID')['ScoreMargin'].mean().reset_index(name='AvgScoreMargin')

# Merge features with team IDs in tournament results for winning and losing teams
combined_tournament_results = pd.concat([tournament_results_men, tournament_results_women])
combined_tournament_results = combined_tournament_results.merge(win_rates, left_on='WTeamID', right_on='WTeamID', how='left')
combined_tournament_results = combined_tournament_results.merge(avg_score_margin, left_on='WTeamID', right_on='WTeamID', how='left')

# Prepare a combined dataset that includes both perspectives (winning and losing)
combined_tournament_results['Outcome'] = 1  # Winning perspective
combined_tournament_results_loss = combined_tournament_results.copy()
#combined_tournament_results_loss['WTeamID'], combined_tournament_results_loss['LTeamID'] = combined_tournament_results_loss['LTeamID'], combined_tournament_results_loss['WTeamID']
combined_tournament_results_loss[['WTeamID', 'LTeamID']] = combined_tournament_results_loss[['LTeamID', 'WTeamID']]

combined_tournament_results_loss['Outcome'] = 0  # Losing perspective

In [10]:
combined_data = pd.concat([combined_tournament_results, combined_tournament_results_loss])
combined_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WinRate,AvgScoreMargin,Outcome
0,1985,136,1116,63,1234,54,N,0,0.640032,16.326608,1
1,1985,136,1120,59,1345,58,N,0,0.548739,13.891271,1
2,1985,136,1207,68,1250,43,N,0,0.605646,14.690678,1
3,1985,136,1229,58,1425,55,N,0,0.534728,10.755869,1
4,1985,136,1242,49,1325,38,N,0,0.805444,16.736581,1


In [11]:
recent_season = combined_data[(combined_data['Season'] == 2023) | (combined_data['Season'] == 2022) | (combined_data['Season'] == 2021)]
recent_season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WinRate,AvgScoreMargin,Outcome
2251,2021,136,1179,53,1455,52,N,0,0.456971,11.122411,1
2252,2021,136,1313,54,1111,53,N,0,0.498731,10.376590,1
2253,2021,136,1411,60,1291,52,N,0,0.474783,11.177656,1
2254,2021,136,1417,86,1277,80,N,1,0.678161,13.468523,1
2255,2021,137,1116,85,1159,68,N,0,0.640032,16.326608,1


In [12]:
# features = ['WinRate', 'AvgScoreMargin']
#X = season_2023.drop(['Outcome', 'WLoc','DayNum', 'WScore','LScore', 'NumOT'], axis=1)
#y = season_2023['Outcome']
X= recent_season.drop(['Season','Outcome', 'WLoc','DayNum', 'WScore','LScore', 'NumOT'], axis=1)
y = recent_season['Outcome']
X

,WTeamID,LTeamID,WinRate,AvgScoreMargin
2251,1179,1455,0.456971,11.122411
2252,1313,1111,0.498731,10.376590
2253,1411,1291,0.474783,11.177656
2254,1417,1277,0.678161,13.468523
2255,1116,1159,0.640032,16.326608
...,...,...,...,...
4029,3268,3376,0.689356,21.901257
4030,3326,3439,0.595745,17.012605
4031,3376,3234,0.660074,15.764045
4032,3439,3261,0.701493,20.054965


In [13]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Logistic Regression
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_scaled, y_train)
y_pred_log_reg = log_reg_model.predict(X_test_scaled)
print("Logistic Regression - Accuracy:", accuracy_score(y_test, y_pred_log_reg))


Logistic Regression - Accuracy: 0.41509433962264153


In [15]:
# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest - Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest - Accuracy: 0.7547169811320755


In [16]:
#Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
print("Gradient Boosting - Accuracy:", accuracy_score(y_test, y_pred_gb))


Gradient Boosting - Accuracy: 0.6729559748427673


In [17]:
# KNN
knn_model = KNeighborsClassifier(n_neighbors=4)
knn_model.fit(X_train_scaled, y_train)
y_pred_knn = knn_model.predict(X_test_scaled)
print("K-NN - Accuracy:", accuracy_score(y_test, y_pred_knn))

K-NN - Accuracy: 0.5786163522012578


In [18]:
combined_tourney_slots_2024 = pd.concat([tournament_slots_men[tournament_slots_men['Season']==2024],tournament_slots_women[tournament_slots_women['Season']==2024]])
combined_tourney_slots_2024 = combined_tourney_slots_2024[combined_tourney_slots_2024['Slot'].str.startswith('R')]
combined_tourney_slots_2024.head()


,Season,Slot,StrongSeed,WeakSeed
2452,2024,R1W1,W01,W16
2453,2024,R1W2,W02,W15
2454,2024,R1W3,W03,W14
2455,2024,R1W4,W04,W13
2456,2024,R1W5,W05,W12


In [19]:
def predict_matchup(team1_id, team2_id, rf_model, features_df):
    # Extract features for team1 and team2
    team1_row = features_df.loc[features_df['TeamID'] == team1_id]
    team2_row = features_df.loc[features_df['TeamID'] == team2_id]

    # Create a DataFrame with the same structure as the one used in training
    # The order of the columns should be the same as during the model training
    matchup_features = pd.DataFrame({
        'WTeamID': team1_row['TeamID'].values,  # Assuming team1 is treated as WTeam
        'LTeamID': team2_row['TeamID'].values,  # Assuming team2 is treated as LTeam
        'WinRate': team1_row['WinRate'].values,  # Assuming the WinRate of WTeam is used
        'AvgScoreMargin': team1_row['AvgScoreMargin'].values  # Assuming the AvgScoreMargin of WTeam is used
    })
    
    # Predict the outcome using the rf_model
    prediction = rf_model.predict(matchup_features)
    
    return team1_id if prediction == 1 else team2_id


In [20]:
# Calculate win rates
win_counts = combined_season_results.groupby('WTeamID').size().reset_index(name='wins')
loss_counts = combined_season_results.groupby('LTeamID').size().reset_index(name='losses')

# Ensure the team ID columns are uniformly named for merging
win_counts.rename(columns={'WTeamID': 'TeamID'}, inplace=True)
loss_counts.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

total_games = pd.merge(win_counts, loss_counts, on='TeamID', how='outer').fillna(0)
total_games['total_games'] = total_games['wins'] + total_games['losses']
total_games['WinRate'] = total_games['wins'] / total_games['total_games']

# Calculate average score margins
win_margins = combined_season_results.groupby('WTeamID').agg(win_margin=('ScoreMargin', 'mean')).reset_index()
win_margins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# For loss margins, calculate the mean difference directly. Note that 'ScoreMargin' needs to be calculated if not already
loss_margins = combined_season_results.groupby('LTeamID').agg(loss_margin=('ScoreMargin', 'mean')).reset_index()
loss_margins['loss_margin'] = -loss_margins['loss_margin']  # Invert the margin for losses
loss_margins.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# Merge win and loss margins and calculate the average
margins = pd.merge(win_margins, loss_margins, on='TeamID', how='outer').fillna(0)
margins['AvgScoreMargin'] = (margins['win_margin'] + margins['loss_margin']) / 2

# Merge win rates and average score margins
features_df = pd.merge(total_games[['TeamID', 'WinRate']], margins[['TeamID', 'AvgScoreMargin']], on='TeamID', how='outer').fillna(0)

In [21]:
seed_to_team_id = dict(zip(tourney_seeds_2024['Seed'], tourney_seeds_2024['TeamID']))

In [22]:
winners = {}  # Dictionary to track winners of each slot

for index, row in combined_tourney_slots_2024.iterrows():
    slot = row['Slot']
    strong_seed = row['StrongSeed']
    weak_seed = row['WeakSeed']
    
    team1_id = seed_to_team_id[strong_seed] if strong_seed in seed_to_team_id else winners[strong_seed]
    team2_id = seed_to_team_id[weak_seed] if weak_seed in seed_to_team_id else winners[weak_seed]
    
    winning_team_id = predict_matchup(team1_id, team2_id, rf_model, features_df)
    
    winners[slot] = winning_team_id
    

In [23]:
team_id_to_seed = {team_id: seed for seed, team_id in seed_to_team_id.items()}

submission_data = []
for index, row in combined_tourney_slots_2024.iterrows():
    slot = row['Slot']
    if slot.startswith('R'):
        team_id = winners.get(slot, '')
        seed = team_id_to_seed.get(team_id, '')
        submission_data.append(['M', 1, slot, seed])

submission_df = pd.DataFrame(submission_data, columns=['Tournament', 'Bracket', 'Slot', 'Team'])
submission_df.insert(0, 'RowId', range(1, len(submission_df) + 1))

women_index = len(submission_df)//2
submission_df.loc[women_index:, 'Tournament'] = 'W'
    
submission_df.to_csv('submission.csv', index=False)